# Semisupervised classification on CWRU dataset

## Configurations

In [1]:
import torch
import numpy as np

In [2]:
torch.cuda.set_device(0)
torch.manual_seed(13)
torch.cuda.manual_seed(13)
torch.backends.cudnn.deterministic=True
np.random.seed(13)

In [3]:
RAW_PATH = './data/raw'
NP_PATH = './data/np'
SAVE_PATH = './saved_models'

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
MASK_P = 0.005  # 0.5% labels are given
SCREEN = False  # screen some combination of (label, diameter) if set as true
SCREEN_LABEL = 1
SCREEN_DIAM = 2

In [6]:
LR = 1e-4
BATCH = 32
FRAME_LEN = 1024
TOL = 100
EPOCH = 10000

PRETRAIN = True
PRETRAIN_EPOCH = 10000
PRE_LR = 1e-4

BETA1=1
BETA2=100
BETA3=100

## Data Preprocessing

In [7]:
from scipy import io
from glob import glob
import os

In [8]:
def find_diameter(n):
    if n<=100:
        return 0
    elif n<169:
        return 1
    elif n<209:
        return 2
    elif n<3000:
        return 3
    else:
        return 4

In [ ]:
def _find_diameter(n):
    if n<=100:
        return 0
    elif n<169:
        return 1
    elif n<209:
        return 2
    elif n<3000:
        return 3
    else:
        return 4
def _load_preprocessed(np_path):
    DATA = np.load('%s/data.npy'%np_path)
    train_idx = np.load('%s/train_idx.npy'%np_path)
    test_idx = np.load('%s/test_idx.npy'%np_path)
    trainY = np.load('%s/train_labels.npy'%np_path)
    testY = np.load('%s/test_labels.npy'%np_path)
    train_diameter = np.load('%s/train_diameter.npy'%np_path)
    test_diameter = np.load('%s/test_diameter.npy'%np_path)
    train_rpm = np.load('%s/train_rpm.npy'%np_path)
    test_rpm = np.load('%s/test_rpm.npy'%np_path)
    return DATA[:,:1], train_idx, test_idx, trainY, testY, train_diameter, test_diameter, train_rpm, test_rpm
def _preprocess(raw_path, out_path, frame_len=1024, frame_intv=512):
    dirnames = glob('%s/*'%raw_path)
    dirnames.sort()
    data = {}
    data['DE']=[]
    data['FE']=[]
    data['BA']=[]
    train_idcs=[]
    test_idcs=[]
    train_labels=[]
    test_labels=[]
    train_rpms=[]
    test_rpms=[]
    cnt=0
    RPM_LIST=[1797,1772,1750,1730]
    train_fault_scale=[]
    test_fault_scale=[]
    for dir in dirnames:
        l = int(os.path.basename(dir)[0])
        fnames = glob(dir+'/*.mat')
        fnames.sort()
        for f in fnames:    
            f_id = int(os.path.basename(f).split('.')[0])
            fault = _find_diameter(f_id)
            mat = io.loadmat(f)
            r=0
            s=0
            fl={}
            for m in ['DE','FE','BA']:
                fl[m]=False
            for k in mat:
                if k[-3:] == 'RPM':
                    r=mat[k][0,0]
                if k[-4:] == 'time' and k[:4] == 'X098' and 'X%03d'%f_id=='X099':
                    continue
                if k[-4:]=='time':
                    data[k[5:7]].append(mat[k])
                    fl[k[5:7]]=True
                    if s>0 and s!= mat[k].shape[0]:
                        print("This is NOT expected")

                    s=mat[k].shape[0]
            for m in ['DE','FE','BA']:
                if not fl[m]:
                    data[m].append(np.zeros((s,1)))
            idx = np.arange(cnt, cnt+s-frame_len, frame_intv)

            train_idx,test_idx = np.split(idx, [(idx.shape[0]*4)//5])
            train_idcs.append(train_idx)
            test_idcs.append(test_idx)
            train_labels.append(np.ones_like(train_idx)*l)
            test_labels.append(np.ones_like(test_idx)*l)
            train_fault_scale.append(np.ones_like(train_idx)*fault)
            test_fault_scale.append(np.ones_like(test_idx)*fault)
            if r==0:
                r=RPM_LIST[(f_id-1)%4]
            trr = np.ones(train_idx.shape[0])*r
            train_rpms.append(trr)
            test_rpms.append(np.ones_like(test_idx)*r)
            cnt+=s
    for d in data:
        data[d] = np.concatenate(data[d],axis=0)
    X=np.concatenate(list(data.values()),axis=-1)
    train_indices = np.concatenate(train_idcs)
    test_indices = np.concatenate(test_idcs)
    train_rpms = np.concatenate(train_rpms)
    test_rpms = np.concatenate(test_rpms)
    train_labels = np.concatenate(train_labels)
    test_labels = np.concatenate(test_labels)
    train_fault_scale = np.concatenate(train_fault_scale)
    test_fault_scale = np.concatenate(test_fault_scale)
    np.save('%s/data.npy'%out_path,X)
    np.save('%s/train_idx.npy'%out_path,train_indices)
    np.save('%s/test_idx.npy'%out_path,test_indices)
    np.save('%s/train_rpm.npy'%out_path,train_rpms)
    np.save('%s/test_rpm.npy'%out_path,test_rpms)
    np.save('%s/train_labels.npy'%out_path, train_labels)
    np.save('%s/test_labels.npy'%out_path, test_labels)
    np.save('%s/train_diameter.npy'%out_path, train_fault_scale)
    np.save('%s/test_diameter.npy'%out_path, test_fault_scale)
    return X[:,:1], train_indices, test_indices, train_labels, test_labels, train_fault_scale, test_fault_scale, train_rpms, test_rpms

def get_preprocessed_data(raw_path, np_path, frame_len, frame_intv):
    '''
    If data has already been preprocessed, load preprocessed data.
    else, read and preprocess data.
    '''
    npy_files = glob("%s/*.npy"%np_path)
    npy_files = [os.path.basename(x) for x in npy_files]
    if set(['data.npy','train_idx.npy','test_idx.npy','train_rpm.npy','test_rpm.npy','train_labels.npy','test_labels.npy','train_diameter.npy','test_diameter.npy']).issubset(set(npy_files)):
        return _load_preprocessed(np_path)
    else:
        return _preprocess(raw_path, np_path, frame_len, frame_intv)

In [9]:
def preprocess_cwru(raw_path, out_path):
    get_preprocessed_data(raw_path, out_path, frame_len=1024, frame_intv=204)

In [10]:
preprocess_cwru(RAW_PATH, NP_PATH)

## Load Data
DATA : (total len of data, 3)
    - each column corresponds to DE/FE/BA data
train_idx/test_idx : (# of frames)
    - index of starting points of frames(1 frame : 1024 points)
trainY/testY : (# of frames)
    - labels(0: normal, 1: inner, 2: ball, 3: outer_centered, 4: outer_orthogonal, 5: outer_opposite)
train_diameter/test_diameter : (# of frames)
    - fault diameters

In [11]:
DATA = np.load('%s/data.npy'%NP_PATH)
train_idx = np.load('%s/train_idx.npy'%NP_PATH)
test_idx = np.load('%s/test_idx.npy'%NP_PATH)
trainY = np.load('%s/train_labels.npy'%NP_PATH)
testY = np.load('%s/test_labels.npy'%NP_PATH)
train_diameter = np.load('%s/train_diameter.npy'%NP_PATH)
test_diameter = np.load('%s/test_diameter.npy'%NP_PATH)

In [12]:
print(DATA.shape, train_idx.shape, test_idx.shape, trainY.shape, testY.shape, train_diameter.shape, test_diameter.shape)

(9012305, 3) (35082,) (8803,) (35082,) (8803,) (35082,) (8803,)


## Define MASK

In [13]:
MASK = np.random.choice(trainY.shape[0], trainY.shape[0], replace=False) < int(MASK_P*trainY.shape[0])

In [14]:
if SCREEN:
    #TBD : masked ratio is not maintained
    MASK[(trainY==SCREEN_LABEL)&(train_diameter==SCREEN_DIAM)] = False

## Train

In [15]:
import torch.nn as nn
import torch.optim as optim
import time

In [16]:
train_shuffle = np.arange(train_idx.shape[0])
if PRETRAIN:
    pretrain_shuffle = np.arange(train_idx[MASK].shape[0])
np.random.shuffle(train_shuffle)
np.random.shuffle(pretrain_shuffle)

In [17]:
from models.model_jh import ENC
param = {
    'in_dim' : 1,
    'h_dim' : 32,
    'n_mode' : 3,
    'enc_depth' : 4,
    'n_label' : 6
}
model = ENC(**param).cuda()

In [18]:
optimizer = optim.Adam(model.parameters(), lr=LR)
if PRETRAIN:
    optimizer_pretrain = optim.Adam(model.parameters(), lr=PRE_LR)
best_eval = float('inf')

In [19]:
def batch_step(model, X, Y, M=None, train=False, optimizer=None):
    if M is None:
        M = torch.zeros_like(Y) == 0
        M = M.to(device=DEVICE, dtype=torch.bool)
    
    m = X.sum(dim=-1) != 0
    if train:
        model.zero_grad()
    recon_loss, class_loss, entropy_loss, pred = model(X,m,Y,M)
    correct = (Y[Y==pred]).shape[0]
    wrong = (Y[Y!=pred]).shape[0]
    
    total_loss = BETA1*recon_loss + BETA2*class_loss + BETA3*entropy_loss
    if train:
        total_loss.backward()
        optimizer.step()
    return total_loss, correct, wrong

### Pretrain (if set as true)

In [20]:
def train(model,optimizer,train_idx,trainY,shuffle,epoch,pretrain=False):
    patience = 0
    best_eval = float('inf')
    for e in range(epoch):
        train_loss = 0.0
        n = 0
        train_correct = 0
        train_wrong = 0
        model.train()
        timestamp = time.time()
        for b in range((shuffle.shape[0]-1)//BATCH+1):
            X = DATA[train_idx[shuffle[b*BATCH:(b+1)*BATCH]][:,None]+np.arange(FRAME_LEN)]
            Y = trainY[shuffle[b*BATCH:(b+1)*BATCH]]
            X = np.transpose(X, [0,2,1])
            X = torch.tensor(X, device=DEVICE, dtype=torch.float)
            Y = torch.tensor(Y, device=DEVICE, dtype=torch.long)
            M=None
            if not pretrain:
                M = MASK[shuffle[b*BATCH:(b+1)*BATCH]]
                M = torch.tensor(M, device=DEVICE, dtype=torch.bool)
            loss, correct, wrong = batch_step(model, X, Y, train=True, optimizer=optimizer)
            train_loss = (train_loss*n + loss.item()*X.shape[0])/(n+X.shape[0])
            n += X.shape[0]
            train_correct += correct
            train_wrong += wrong
        model.eval()
        eval_loss = 0.0
        n = 0
        eval_correct = 0
        eval_wrong = 0
        with torch.no_grad():
            for b in range((test_idx.shape[0]-1)//BATCH+1):
                X = DATA[test_idx[b*BATCH:(b+1)*BATCH][:,None]+np.arange(FRAME_LEN)]
                Y = testY[b*BATCH:(b+1)*BATCH]
                X = np.transpose(X, [0,2,1])
                X = torch.tensor(X, device=DEVICE, dtype=torch.float)
                Y = torch.tensor(Y, device=DEVICE, dtype=torch.long)

                loss, correct, wrong = batch_step(model, X, Y, train=False)
                eval_loss = (eval_loss*n + loss.item()*X.shape[0])/(n+X.shape[0])
                n+= X.shape[0]
                eval_correct += correct
                eval_wrong += wrong
        mode = 'Pretrain' if pretrain else 'Train'
        print('(%.2fs)/[%s %d]\n\t(train) loss : %.5f,\tacc : %.5f'%(time.time()-timestamp, mode,e+1, train_loss, train_correct/(train_correct+train_wrong)))
        print('\t(eval) loss : %.5f,\tacc : %.5f'%(eval_loss, eval_correct/(eval_correct+eval_wrong)))
        if eval_loss < best_eval:
            best_eval = eval_loss
            patience = 0
            torch.save(model.state_dict(), '%s/%s_best.pth'%(SAVE_PATH,mode))
        patience += 1
        if patience>TOL:
            print('Early stop at Epoch %d'%(e+1))
            break

In [21]:
if PRETRAIN:
    pretrain_param = {
        'model' : model,
        'shuffle' : pretrain_shuffle,
        'epoch' : PRETRAIN_EPOCH,
        'pretrain' : True,
        'train_idx' : train_idx[MASK],
        'trainY' : trainY[MASK],
        'optimizer' : optimizer_pretrain
    }
    train(**pretrain_param)
    model.load_state_dict(torch.load('%s/pretrain_best.pth'%SAVE_PATH))

(9.06s)/[Pretrain 1]
	(train) loss : 209.12918,	acc : 0.20571
	(eval) loss : 209.12376,	acc : 0.22436
(7.32s)/[Pretrain 2]
	(train) loss : 209.06720,	acc : 0.27429
	(eval) loss : 209.07051,	acc : 0.21652
(6.29s)/[Pretrain 3]
	(train) loss : 209.00255,	acc : 0.26857
	(eval) loss : 209.00995,	acc : 0.21595
(6.19s)/[Pretrain 4]
	(train) loss : 208.93387,	acc : 0.26857
	(eval) loss : 208.94321,	acc : 0.21413
(5.98s)/[Pretrain 5]
	(train) loss : 208.85924,	acc : 0.25143
	(eval) loss : 208.87182,	acc : 0.21493
(6.10s)/[Pretrain 6]
	(train) loss : 208.78052,	acc : 0.28000
	(eval) loss : 208.79726,	acc : 0.21470
(6.10s)/[Pretrain 7]
	(train) loss : 208.70025,	acc : 0.26286
	(eval) loss : 208.71558,	acc : 0.21686
(6.06s)/[Pretrain 8]
	(train) loss : 208.61156,	acc : 0.28000
	(eval) loss : 208.62921,	acc : 0.29433
(6.10s)/[Pretrain 9]
	(train) loss : 208.52159,	acc : 0.28000
	(eval) loss : 208.53663,	acc : 0.26127
(6.13s)/[Pretrain 10]
	(train) loss : 208.42587,	acc : 0.27429
	(eval) loss : 208.

(6.93s)/[Pretrain 81]
	(train) loss : 138.17201,	acc : 0.44000
	(eval) loss : 133.55121,	acc : 0.42542
(6.32s)/[Pretrain 82]
	(train) loss : 138.58436,	acc : 0.42857
	(eval) loss : 132.41676,	acc : 0.46064
(6.47s)/[Pretrain 83]
	(train) loss : 135.79416,	acc : 0.53143
	(eval) loss : 131.48564,	acc : 0.48506
(6.62s)/[Pretrain 84]
	(train) loss : 138.80917,	acc : 0.44000
	(eval) loss : 130.16546,	acc : 0.45394
(6.37s)/[Pretrain 85]
	(train) loss : 135.47028,	acc : 0.45143
	(eval) loss : 129.99463,	acc : 0.45791
(6.31s)/[Pretrain 86]
	(train) loss : 134.63345,	acc : 0.49143
	(eval) loss : 130.40325,	acc : 0.48109
(6.15s)/[Pretrain 87]
	(train) loss : 133.26303,	acc : 0.54857
	(eval) loss : 128.52762,	acc : 0.49665
(6.09s)/[Pretrain 88]
	(train) loss : 135.72044,	acc : 0.47429
	(eval) loss : 127.19855,	acc : 0.49869
(6.01s)/[Pretrain 89]
	(train) loss : 131.04488,	acc : 0.53714
	(eval) loss : 128.43711,	acc : 0.52721
(6.15s)/[Pretrain 90]
	(train) loss : 134.15120,	acc : 0.48571
	(eval) lo

FileNotFoundError: [Errno 2] No such file or directory: './saved_models/Pretrain_best.pth'

In [ ]:
train_param = {
    'model' : model,
    'shuffle' : train_shuffle,
    'epoch' : EPOCH,
    'pretrain' : False,
    'train_idx' : train_idx,
    'trainY' : trainY,
    'optimizer' : optimizer
}
train(**train_param)

## Test